In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rsome import ro 
import rsome as rso                           # import the ro module
from rsome import grb_solver as grb
from torch.utils.data import DataLoader, TensorDataset
import sbibm
import model

/home/srayan/miniconda3/envs/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This is the setup for vehicle pre-allocation problem but it looks like they have some 
# wait and see decisions to make.  I'm not sure how to handle that.  I'm going to try 
# (the last two sentences are from github copilot! Wild!)
I, J = 1, 10
r = np.array([4.50, 4.41, 3.61, 4.49, 4.38, 4.58, 4.53, 4.64, 4.58, 4.32])
c = 3 * np.ones((I, J))
q = 400 * np.ones(I)

In [51]:
# Shortest path problem or minimum cost flow problem
# min_{x} max_{c in U(c)} c^Tx
# Constraints x[i, j]

# Creating the edge list so that one can only travel south or west

edges = np.zeros((25, 25))
edge_list = {}
adjacency_list = {i : [[], []] for i in range(25)}
grid_size = 5
vertex_count = 25

edge_index = 0
for i in range(grid_size):
    for j in range(grid_size):
        if i <(grid_size -1) and j < (grid_size -1):
            # East edge
            edges[i * 5 + j, i * 5 + j + 1] = 1
            # edge_list[edge_index] = (i * 5 + j, i * 5 + j + 1)
            edge_list[(i * 5 + j, i * 5 + j + 1)] = edge_index
            edge_index += 1
            adjacency_list[i * 5 + j][0].append(i * 5 + j + 1)
            adjacency_list[i * 5 + j + 1][1].append(i * 5 + j)

            # South edge
            edges[i * 5 + j, (i + 1) * 5 + j] = 1
            # edge_list[edge_index] = (i * 5 + j, (i + 1) * 5 + j)
            edge_list[(i * 5 + j, (i + 1) * 5 + j)] = edge_index
            edge_index += 1
            adjacency_list[i * 5 + j][0].append((i + 1) * 5 + j)
            adjacency_list[(i + 1) * 5 + j][1].append(i * 5 + j)
        elif i== (grid_size -1) and j < (grid_size -1):
            # East edge
            edges[i * 5 + j, i * 5 + j + 1] = 1
            # edge_list[edge_index] = (i * 5 + j, i * 5 + j + 1)
            edge_list[(i * 5 + j, i * 5 + j + 1)] = edge_index
            edge_index += 1
            adjacency_list[i * 5 + j][0].append(i * 5 + j + 1)
            adjacency_list[i * 5 + j + 1][1].append(i * 5 + j)

        elif i < (grid_size -1) and j == (grid_size -1):
            # South edge
            edges[i * 5 + j, (i + 1) * 5 + j] = 1
            # edge_list[edge_index] = (i * 5 + j, (i + 1) * 5 + j)
            edge_list[(i * 5 + j, (i + 1) * 5 + j)] = edge_index
            edge_index += 1
            adjacency_list[i * 5 + j][0].append((i + 1) * 5 + j)
            adjacency_list[(i + 1) * 5 + j][1].append(i * 5 + j)
edge_count = int(edges.sum())
rev_edge_list = {v : k for k, v in edge_list.items()}

In [6]:
def generate_data(N, d, Theta, edge_count):
    # Generating zs
    z = np.random.normal(0, 1, (N, d))

    # Computing cost with some noise
    cost_wo_noise = ((1/d)**(0.5) * np.matmul(z, Theta.T) + 3)**5 + 1
    noise = np.random.uniform(low = 0.75, high = 1.25, size = (N, edge_count))
    cost = cost_wo_noise * noise
    return z, cost 

In [7]:
# Data generation for training and calibration
# generating the theta such that some of the covariates are independent of cost
np.random.seed(0)
d = 10
Theta = np.random.binomial(1, 0.5, (edge_count, d))
irrelevant_zs = np.random.choice(range(d), size = 2)
Theta[:, irrelevant_zs] = 0
N_train = 1000
z_train, cost_train = generate_data(N_train, d, Theta, edge_count)
N_calib = 500
z_calib, cost_calib = generate_data(N_calib, d, Theta, edge_count)
N_test = 1000
z_test, c_test = generate_data(N_test, d, Theta, edge_count)

In [8]:
# Creating the dataset for the model
device = ("cuda" if torch.cuda.is_available() else "cpu")
to_tensor = lambda r : torch.tensor(r).to(torch.float32).to(device)
z_train, z_cal = to_tensor(z_train), to_tensor(z_calib)
c_train, c_cal = to_tensor(cost_train), to_tensor(cost_calib)
z_test, c_test = to_tensor(z_test), to_tensor(c_test)
train_dataset = TensorDataset(z_train, c_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from collections import OrderedDict

class MLP(nn.Module):
    def __init__(self, hidden_sizes):
        super(MLP, self).__init__()
        order_dict = []
        for i in range(len(hidden_sizes) - 1):
            order_dict.append(('Linear Layer {}'.format(i), nn.Linear(hidden_sizes[i], hidden_sizes[i+1])))
            if i < len(hidden_sizes) - 2:
                order_dict.append(('BatchNorm Layer {}'.format(i), nn.BatchNorm1d(hidden_sizes[i+1])))
                order_dict.append(('ReLU Layer {}'.format(i), nn.ReLU()))
        self.mlp = nn.Sequential(OrderedDict(order_dict))
    
    def forward(self, x):
        return self.mlp(x)

In [10]:
model = MLP([d, 64, 128, 64, edge_count]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1_000

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(batch_X)  # Forward pass
        loss = criterion(outputs, batch_y)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/1000], Loss: 264677.8750
Epoch [101/1000], Loss: 104702.0703
Epoch [201/1000], Loss: 57920.2383
Epoch [301/1000], Loss: 45233.1445
Epoch [401/1000], Loss: 27315.7480
Epoch [501/1000], Loss: 86438.4219
Epoch [601/1000], Loss: 42735.6562
Epoch [701/1000], Loss: 15530.2734
Epoch [801/1000], Loss: 13807.7217
Epoch [901/1000], Loss: 12944.4492


In [11]:
def intercept_model_train(c_train):
    return c_train.mean(0)

marg_model = intercept_model_train(c_train)

In [12]:
def box_score_quantile(alpha, model, c_calib, isCond = False, z_calib = None):
    if not isCond:
        preds = model
    else:
        model.eval()
        preds = model(z_calib)
    with torch.no_grad():
        losses = np.linalg.norm((preds - c_calib).detach().cpu().numpy(), np.inf, axis=1) 
        N_c = c_calib.shape[0]
        return np.quantile(losses, (N_c + 1)*(1 - alpha)/N_c) #, np.sort(losses)

In [15]:
def ellipsoid_score_quantile(alpha, model, c_calib, isCond= False, z_calib = None):
    if not isCond:
        preds = model
    else:
        model.eval()
        preds = model(z_calib)
    with torch.no_grad():
        residuals = (preds - c_calib).detach().cpu().numpy()
        cov = np.cov(residuals.T)
        N_c = c_calib.shape[0]
        calib_scores = np.sqrt(np.einsum('ni ,ij, jn -> n', residuals, np.linalg.inv(cov), residuals.T))
        return np.quantile(calib_scores, (N_c + 1)*(1 - alpha)/N_c), cov



In [43]:
def diamond_score_quantile(alpha, model, c_calib, isCond= False, z_calib = None):
    if not isCond:
        preds = model
    else:
        model.eval()
        preds = model(z_calib)
    with torch.no_grad():
        losses = np.linalg.norm((preds - c_calib).detach().cpu().numpy(), ord=1, axis=1) 
        N_c = c_calib.shape[0]
        return np.quantile(losses, (N_c + 1)*(1 - alpha)/N_c)


In [44]:
score = box_score_quantile(0.05, model, c_cal, True, z_cal)
score_marg = box_score_quantile(0.05, marg_model, c_cal)
score_elip, cov_elip = ellipsoid_score_quantile(0.05, model, c_cal, True, z_cal)
score_elip_marg, cov_elip_marg = ellipsoid_score_quantile(0.05, marg_model, c_cal)
score_diam = diamond_score_quantile(0.05, model, c_cal, True, z_cal)
score_diam_marg = diamond_score_quantile(0.05, marg_model, c_cal)

In [17]:
def fill_cost_matrix(cost_vector, edges, vector_count):
    cost_matrix = np.zeros((vector_count, vector_count))
    cost_index = 0
    for i in range(vector_count):
        for j in range(vector_count):
            if edges[i, j] == 1:
                cost_matrix[i, j] = cost_vector[cost_index]
                cost_index += 1
            else:
                cost_matrix[i, j] = 1000000
    return cost_matrix

In [18]:
def no_uq_solve(model, edge_list, adjacency_list, z0, vertex_count, edge_count):
    c_pred = model(z0)
    c_pred = c_pred.detach().cpu().numpy().reshape(-1)
    # c_pred = fill_cost_matrix(c_pred, edges, vertex_count)

    model = ro.Model()
    w = model.dvar(edge_count, 'B')
    model.min((c_pred*w).sum())
    model.st(w[0] + w[1] == 1)
    model.st(w[-1] + w[-6] == 1)
    for j in range(1, vertex_count - 1):
        outgoing_edges = [(j, i) for i in adjacency_list[j][0]]
        incoming_edges = [(i, j) for i in adjacency_list[j][1]]
        model.st(w[[edge_list.get(x) for x in outgoing_edges]].sum() - w[[edge_list.get(x) for x in incoming_edges]].sum() == 0)
    model.solve(grb)
    return model.get(), w.get()

In [21]:
# Conditonal/marginal box solve
def box_solve_RO(model, CP_score, edge_list, adjacency_list, vertex_count, edge_count, c_true, isCond = False, z0 = None):

    # Checking if the model is conditional or marginal
    if not isCond:
        c_pred = model.detach().cpu().numpy().reshape(-1)
    else:
        c_pred = model(z0)
        c_pred = c_pred.detach().cpu().numpy().reshape(-1)
    
    # Computing the lower and upper bounds for the Uncertainty set
    c_lb = c_pred - CP_score
    c_ub = c_pred + CP_score

    # Checking if the true cost is in the uncertainty set
    truth = c_true.detach().cpu().numpy().reshape(-1)
    covered = (truth >= c_lb).all() and (truth <= c_ub).all()

    # Initializing the model, decision variables and random variables
    model = ro.Model()
    w = model.dvar(edge_count, 'B')
    c = model.rvar(edge_count)
    uset = (c_lb <= c, c <= c_ub)

    # Adding the objective and constraints
    model.minmax((c*w).sum(), uset)
    model.st(w[0] + w[1] == 1)
    model.st(w[-1] + w[-6] == 1)
    for j in range(1, vertex_count - 1):
        outgoing_edges = [(j, i) for i in adjacency_list[j][0]]
        incoming_edges = [(i, j) for i in adjacency_list[j][1]]
        model.st(w[[edge_list.get(x) for x in outgoing_edges]].sum() - w[[edge_list.get(x) for x in incoming_edges]].sum() == 0)
    model.solve(grb)
    return model.get(), w.get(), covered

In [45]:
# Conditional/marginal ellipsoid solve
def ellipsoid_solve_RO(model, CP_score, cov,  edge_list, adjacency_list,  vertex_count, edge_count, c_true, isCond = False, z0 = None):
    # Checking if the model is conditional or marginal
    if not isCond:
        c_pred = model.detach().cpu().numpy().reshape(-1)
    else:
        c_pred = model(z0)
        c_pred = c_pred.detach().cpu().numpy().reshape(-1)

     # Checking if the true cost is in the uncertainty set
    truth = c_true.detach().cpu().numpy().reshape(-1)
    cholesky_sqrt = np.linalg.cholesky(np.linalg.inv(cov))
    covered = (np.linalg.norm((truth - c_pred) @ cholesky_sqrt) <= CP_score)

    # Initializing the model, decision variables and random variables
    model = ro.Model()
    w = model.dvar(edge_count, 'B')
    c = model.rvar(edge_count)
    uset = rso.norm((c - c_pred).T @ cholesky_sqrt) <= CP_score
    print(uset)

    # Adding the objective and constraints
    model.minmax((c*w).sum(), uset)
    model.st(w[0] + w[1] == 1)
    model.st(w[-1] + w[-6] == 1) # specific to 5 by 5 grid
    for j in range(1, vertex_count - 1):
        outgoing_edges = [(j, i) for i in adjacency_list[j][0]]
        incoming_edges = [(i, j) for i in adjacency_list[j][1]]
        model.st(w[[edge_list.get(x) for x in outgoing_edges]].sum() - w[[edge_list.get(x) for x in incoming_edges]].sum() == 0)
    

    # Solving the model
    model.solve(grb)
    return model.get(), w.get(), covered
    
    
    

In [47]:
# Conditonal/marginal diamond solve
def diamond_solve_RO(model, CP_score, edge_list, adjacency_list, vertex_count, edge_count, c_true, isCond = False, z0 = None):

    # Checking if the model is conditional or marginal
    if not isCond:
        c_pred = model.detach().cpu().numpy().reshape(-1)
    else:
        c_pred = model(z0)
        c_pred = c_pred.detach().cpu().numpy().reshape(-1)

    # Checking if the true cost is in the uncertainty set
    truth = c_true.detach().cpu().numpy().reshape(-1)
    covered = (np.linalg.norm(truth - c_pred, ord = 1)<= CP_score)

    # Initializing the model, decision variables and random variables
    model = ro.Model()
    w = model.dvar(edge_count, 'B')
    c = model.rvar(edge_count)
    uset = rso.norm((c - c_pred), degree = 1) <= CP_score

    # Adding the objective and constraints
    model.minmax((c*w).sum(), uset)
    model.st(w[0] + w[1] == 1)
    model.st(w[-1] + w[-6] == 1)
    for j in range(1, vertex_count - 1):
        outgoing_edges = [(j, i) for i in adjacency_list[j][0]]
        incoming_edges = [(i, j) for i in adjacency_list[j][1]]
        model.st(w[[edge_list.get(x) for x in outgoing_edges]].sum() - w[[edge_list.get(x) for x in incoming_edges]].sum() == 0)
    model.solve(grb)
    return model.get(), w.get(), covered

In [22]:
obj_ro, w_ro, covered = box_solve_RO(model, score, edge_list, adjacency_list, 25, 40, c_test[0], True, z_test[0, :].reshape(1, -1))
obj_ro_marg, w_ro_marg, covered_marg = box_solve_RO(marg_model, score_marg, edge_list, adjacency_list, 25, 40, c_test[0])

Being solved by Gurobi...
Solution status: 2
Running time: 0.0009s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0009s


In [46]:
obj_ro_ellip, w_ro_ellip, covered_ellip = ellipsoid_solve_RO(model, score_elip, cov_elip, edge_list, adjacency_list,  25, 40, c_test[0], True, z_test[0, :].reshape(1, -1))
obj_ro_ellip_marg, w_ro_ellip_marg, covered_ellip_marg = ellipsoid_solve_RO(marg_model, score_marg, cov_elip_marg, edge_list, adjacency_list,  25, 40, c_test[0])

1 convex constraint
Being solved by Gurobi...
Solution status: 2
Running time: 0.2252s
1 convex constraint
Being solved by Gurobi...
Solution status: 2
Running time: 0.0564s


In [48]:
obj_ro_dia, w_ro_dia, covered_dia = diamond_solve_RO(model, score_diam, edge_list, adjacency_list, 25, 40, c_test[0], True, z_test[0, :].reshape(1, -1))
obj_ro_dia_marg, w_ro_dia_marg, covered_dia_marg = diamond_solve_RO(marg_model, score_diam_marg, edge_list, adjacency_list, 25, 40, c_test[0])

Being solved by Gurobi...
Solution status: 2
Running time: 0.0018s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0013s


In [20]:
obj, w = no_uq_solve(model, edge_list, adjacency_list, z_test[0, :].reshape(1, -1), 25, 40)

Restricted license - for non-production use only - expires 2024-10-28
Being solved by Gurobi...
Solution status: 2
Running time: 0.0010s


In [ ]:
print(model(z_test[0, :].reshape(1, -1)))
print(covered)
print(obj)
print(np.where(w == 1))

In [35]:
print(rev_edge_list)
print(np.argwhere(w_ro == 1).reshape(-1))
print(obj_ro)
print(covered)

print(np.argwhere(w_ro_marg == 1).reshape(-1))
print(obj_ro_marg)
print(covered_marg)

{0: (0, 1), 1: (0, 5), 2: (1, 2), 3: (1, 6), 4: (2, 3), 5: (2, 7), 6: (3, 4), 7: (3, 8), 8: (4, 9), 9: (5, 6), 10: (5, 10), 11: (6, 7), 12: (6, 11), 13: (7, 8), 14: (7, 12), 15: (8, 9), 16: (8, 13), 17: (9, 14), 18: (10, 11), 19: (10, 15), 20: (11, 12), 21: (11, 16), 22: (12, 13), 23: (12, 17), 24: (13, 14), 25: (13, 18), 26: (14, 19), 27: (15, 16), 28: (15, 20), 29: (16, 17), 30: (16, 21), 31: (17, 18), 32: (17, 22), 33: (18, 19), 34: (18, 23), 35: (19, 24), 36: (20, 21), 37: (21, 22), 38: (22, 23), 39: (23, 24)}
[ 0  3 11 14 23 32 38 39]
6462.946044921875
True
[ 1  9 12 20 23 32 38 39]
21248.157470703125
True


In [40]:
print(obj_ro_ellip)
print(covered_ellip)
print(np.argwhere(w_ro_ellip == 1).reshape(-1))

print(obj_ro_ellip_marg)
print(covered_ellip_marg)
print(np.argwhere(w_ro_ellip_marg == 1).reshape(-1))

4561.2061394798275
True
[ 1  9 12 20 23 32 38 39]
4015714.5448763384
True
[ 1  9 12 20 23 32 38 39]


In [50]:
print(rev_edge_list)
print(obj_ro_dia)
print(covered_dia)
print(np.argwhere(w_ro_dia == 1).reshape(-1))

print(obj_ro_dia_marg)
print(covered_dia_marg)
print(np.argwhere(w_ro_dia_marg == 1).reshape(-1))

{0: (0, 1), 1: (0, 5), 2: (1, 2), 3: (1, 6), 4: (2, 3), 5: (2, 7), 6: (3, 4), 7: (3, 8), 8: (4, 9), 9: (5, 6), 10: (5, 10), 11: (6, 7), 12: (6, 11), 13: (7, 8), 14: (7, 12), 15: (8, 9), 16: (8, 13), 17: (9, 14), 18: (10, 11), 19: (10, 15), 20: (11, 12), 21: (11, 16), 22: (12, 13), 23: (12, 17), 24: (13, 14), 25: (13, 18), 26: (14, 19), 27: (15, 16), 28: (15, 20), 29: (16, 17), 30: (16, 21), 31: (17, 18), 32: (17, 22), 33: (18, 19), 34: (18, 23), 35: (19, 24), 36: (20, 21), 37: (21, 22), 38: (22, 23), 39: (23, 24)}
6761.48194529724
True
[ 0  3 11 14 23 32 38 39]
24708.191504101556
True
[ 1  9 12 20 23 32 38 39]
